In [ ]:
# ----------------------------
#  Setup Spark in Colab
!apt-get update -qq
!apt-get install -y openjdk-11-jdk-headless
!wget -q https://archive.apache.org/dist/spark/spark-3.5.1/spark-3.5.1-bin-hadoop3.tgz
!tar -xf spark-3.5.1-bin-hadoop3.tgz
!pip install -q findspark pyspark

import os
import findspark
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.1-bin-hadoop3"
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("CropYieldPrediction") \
    .config("spark.sql.repl.eagerEval.enabled", True) \
    .getOrCreate()

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  openjdk-11-jre-headless
Suggested packages:
  openjdk-11-demo openjdk-11-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  | fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-11-jdk-headless openjdk-11-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 57 not upgraded.
Need to get 116 MB of archives.
After this operation, 258 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 openjdk-11-jre-headless amd64 11.0.29+7-1ubuntu1~22.04 [42.6 MB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64

In [ ]:
# 1️⃣ Load Data
df_spark = spark.read.csv("crop_model_data_with_header.csv", header=True, inferSchema=True)
df_spark.show(5)
df_spark.printSchema()

# ----------------------------
# 2️⃣ Drop previous index/vec columns if re-running
for col_name in ["crop_index", "season_index", "state_index", "crop_vec", "season_vec", "state_vec"]:
    if col_name in df_spark.columns:
        df_spark = df_spark.drop(col_name)

# ----------------------------
# 3️⃣ Optional: Remove extreme outliers
# Filter yields > 100 (adjust threshold if needed)
from pyspark.sql.functions import col

# Remove extreme outliers
df_spark = df_spark.filter(col("yield") <= 100)


# ----------------------------
# 4️⃣ Categorical columns
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, StandardScaler
categorical_cols = ["crop", "season", "state"]

indexers = [StringIndexer(inputCol=col, outputCol=col+"_index", handleInvalid="keep") for col in categorical_cols]
encoders = [OneHotEncoder(inputCol=col+"_index", outputCol=col+"_vec") for col in categorical_cols]

# ----------------------------
# 5️⃣ Feature Engineering
from pyspark.sql.functions import col, log1p, expm1

# Log-transform target
df_spark = df_spark.withColumn("yield_log", log1p("yield"))

# Engineered features
df_spark = df_spark.withColumn("yield_per_area", col("production") / col("area"))
df_spark = df_spark.withColumn("humidity_temp_ratio", col("avg_humidity_percent") / col("avg_temp_c"))
df_spark = df_spark.withColumn("rainfall_ph_interaction", col("total_rainfall_mm") * col("ph"))

# ----------------------------
# 6️⃣ Feature Columns
feature_cols = [
    "year", "area", "production", "fertilizer", "pesticide",
    "nitrogen", "phosphorus", "potassium", "ph",
    "avg_temp_c", "total_rainfall_mm", "avg_humidity_percent",
    "yield_per_area", "humidity_temp_ratio", "rainfall_ph_interaction",
    "crop_vec", "season_vec", "state_vec"
]

# ----------------------------
# 7️⃣ Vector Assembler
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")

# ----------------------------
# 8️⃣ Feature Scaling
scaler = StandardScaler(inputCol="features", outputCol="scaled_features", withMean=True, withStd=True)

# ----------------------------
# 9️⃣ Full Pipeline (indexing + encoding + assembler + scaling)
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=indexers + encoders + [assembler, scaler])
df_spark = pipeline.fit(df_spark).transform(df_spark)


+------------+----+-----------+-----+-------+----------+----------+---------+-----------+--------+----------+---------+---+----------+-----------------+--------------------+
|        crop|year|     season|state|   area|production|fertilizer|pesticide|      yield|nitrogen|phosphorus|potassium| ph|avg_temp_c|total_rainfall_mm|avg_humidity_percent|
+------------+----+-----------+-----+-------+----------+----------+---------+-----------+--------+----------+---------+---+----------+-----------------+--------------------+
|    Arecanut|1997|Whole Year |Assam|73814.0|   56708.0|7024878.38| 22882.34|0.796086957|    60.0|      18.0|     38.0|5.8|     22.41|          1468.92|               70.71|
|   Arhar/Tur|1997|Kharif     |Assam| 6637.0|    4685.0| 631643.29|  2057.47|0.710434783|    60.0|      18.0|     38.0|5.8|     22.41|          1468.92|               70.71|
| Castor seed|1997|Kharif     |Assam|  796.0|      22.0|  75755.32|   246.76|0.238333333|    60.0|      18.0|     38.0|5.8|     22

In [ ]:
# ----------------------------
# 🔹 10  1️⃣Split Data
train_data, test_data = df_spark.randomSplit([0.8, 0.2], seed=42)

In [ ]:

# ----------------------------
# 🔹 11️⃣ Define Models
from pyspark.ml.regression import LinearRegression, RandomForestRegressor, GBTRegressor, DecisionTreeRegressor

models = {
    "Linear Regression": LinearRegression(featuresCol="scaled_features", labelCol="yield_log"),
    "Random Forest": RandomForestRegressor(featuresCol="scaled_features", labelCol="yield_log", numTrees=100),
    "GBT Regressor": GBTRegressor(featuresCol="scaled_features", labelCol="yield_log", maxIter=50),
    "Decision Tree": DecisionTreeRegressor(featuresCol="scaled_features", labelCol="yield_log")
}

# ----------------------------
# 🔹 12️⃣ Train, Predict, Evaluate
from pyspark.ml.evaluation import RegressionEvaluator

evaluator_rmse = RegressionEvaluator(labelCol="yield_log", predictionCol="prediction", metricName="rmse")
evaluator_r2 = RegressionEvaluator(labelCol="yield_log", predictionCol="prediction", metricName="r2")

for name, model in models.items():
    print(f"\n--- {name} ---")
    model_trained = model.fit(train_data)
    predictions = model_trained.transform(test_data)

    # Convert predictions back to original scale
    predictions = predictions.withColumn("prediction_original", expm1("prediction"))

    # Evaluate on log scale
    rmse_log = evaluator_rmse.evaluate(predictions)
    r2_log = evaluator_r2.evaluate(predictions)

    # Evaluate on original scale
    evaluator_rmse_orig = RegressionEvaluator(labelCol="yield", predictionCol="prediction_original", metricName="rmse")
    evaluator_r2_orig = RegressionEvaluator(labelCol="yield", predictionCol="prediction_original", metricName="r2")
    rmse_orig = evaluator_rmse_orig.evaluate(predictions)
    r2_orig = evaluator_r2_orig.evaluate(predictions)

    print(f"RMSE (log scale): {rmse_log:.4f}, R² (log scale): {r2_log:.4f}")
    print(f"RMSE (original scale): {rmse_orig:.4f}, R² (original scale): {r2_orig:.4f}")

    # Show sample predictions
    predictions.select("yield", "prediction_original").show(5)



--- Linear Regression ---
RMSE (log scale): 0.2824, R² (log scale): 0.8861
RMSE (original scale): 10.2053, R² (original scale): -0.0558
+-----------+-------------------+
|      yield|prediction_original|
+-----------+-------------------+
|1.147857143| 1.1332131126154503|
|0.784347826| 1.0860829119578823|
|      6.232|  1.943350459923341|
|       1.21| 1.6157019573615758|
|0.957391304|  1.155544688296508|
+-----------+-------------------+
only showing top 5 rows


--- Random Forest ---
RMSE (log scale): 0.1870, R² (log scale): 0.9500
RMSE (original scale): 3.2053, R² (original scale): 0.8959
+-----------+-------------------+
|      yield|prediction_original|
+-----------+-------------------+
|1.147857143| 1.3008902807835683|
|0.784347826| 0.8821243272408814|
|      6.232|  6.141236681522768|
|       1.21| 0.9922039648462673|
|0.957391304| 0.9732589499624517|
+-----------+-------------------+
only showing top 5 rows


--- GBT Regressor ---
RMSE (log scale): 0.0919, R² (log scale): 0.987